In [41]:
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd
from os.path import join

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from classification import read_generator, IMAGE_HEIGHT, IMAGE_WIDTH, CLASSES_NUM, pad_and_scale
from sklearn.model_selection import train_test_split
from skimage.color import grey2rgb
from tqdm import tqdm_notebook as tqdmn
from skimage.io import imread

In [10]:
%load_ext autoreload
%autoreload 2

In [19]:
pd.read_csv('./public_data/00_input/train/gt.csv')['class_id'].values

,filename,class_id
0,0000.jpg,0
1,0001.jpg,0
2,0002.jpg,0
3,0003.jpg,0
4,0004.jpg,0
5,0005.jpg,0
6,0006.jpg,0
7,0007.jpg,0
8,0008.jpg,0
9,0009.jpg,0


In [28]:
AS_GREY = False
CHANNELS = 1 if AS_GREY else 3

In [21]:
def read_data(path='./public_data/00_input/train/gt.csv', split=0):
    file = pd.read_csv(path)
    
    if not split:
        return {name:label for name, label in file.values}, {}
    
    x = file['filename'].values.ravel()
    y = file['class_id'].values.ravel()
    files_train, files_test, y_train, y_test = train_test_split(x, y, test_size=split, stratify=y)
    
    return \
        {name:label for name, label in zip(files_train, y_train)}, \
        {name:label for name, label in zip(files_test, y_test)}

train_data, val_data = read_data(split=0.2)

In [24]:
len(val_data), len(train_data)

(500, 2000)

In [42]:
def read_test(y, img_dir='./public_data/00_input/train/images/'):
    test = np.zeros(( len(y), IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
    y_test = np.zeros((len(y), CLASSES_NUM))
    
    for i, img_name in tqdmn(enumerate(y.keys()), total=len(y)):
        img = imread(join(img_dir, img_name), as_grey=AS_GREY)
        
        if len(img.shape) == 2 and not AS_GREY:
            img = grey2rgb(img)
            
        img, _ = pad_and_scale(img)
        
        test[i, ...] = img.reshape((IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
        y_test[i] = y[img_name]

    return y_test, test

y_val, x_val = read_test(val_data)

In [43]:
checkpoint_callback = ModelCheckpoint(filepath = 'checkpoint.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
early_callback = EarlyStopping(patience=15)
lr_callback = ReduceLROnPlateau(patience=5)

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(CLASSES_NUM, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(
    read_generator(train_data, './public_data/00_input/train/images', 32, grey=False),  
    steps_per_epoch=len(train_data) // 32, 
    epochs=10,
    callbacks=[checkpoint_callback, early_callback, lr_callback],
    validation_data=(x_val, y_val)
)

Epoch 1/10
62/62 [==============================] - 689s 11s/step - loss: 4835.8144 - val_loss: 4792.2399
Epoch 2/10
17/62 [=======>......................] - ETA: 6:32 - loss: 4665.6665

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)